In [1]:
"""
Create Grid Coordinates for Airbnb Search
==========================================

Simple tool to partition any rectangular area into optimal grid cells.

Usage:
    Just provide NE and SW corners, the script does the rest!

Example:
    python create_chicago_grid.py
"""

import pandas as pd
import numpy as np
from pathlib import Path
import os


def create_grid_from_bounds(
    ne_lat, ne_long, sw_lat, sw_long,
    region_name="Region",
    num_rows=None,
    num_cols=None,
    output_file=None
):
    """
    Create a grid of bounding boxes from a rectangular area.
    
    Simple interface: Just provide the NE and SW corners!
    
    Args:
        ne_lat: Northeast corner latitude
        ne_long: Northeast corner longitude
        sw_lat: Southwest corner latitude
        sw_long: Southwest corner longitude
        region_name: Name for the region (e.g., "Chicago")
        num_rows: Number of grid rows (auto-calculated if None)
        num_cols: Number of grid columns (auto-calculated if None)
        output_file: Where to save (auto-generated if None)
    
    Returns:
        DataFrame with grid coordinates
    """
    
    # Calculate area dimensions
    lat_span = abs(ne_lat - sw_lat)
    lon_span = abs(ne_long - sw_long)
    
    # Auto-calculate optimal grid size if not provided
    # Target: each grid ~0.03-0.05 degrees (~3-5 km) for optimal discovery
    if num_rows is None:
        # Aim for grids of ~0.04 degrees latitude (~4.5 km)
        optimal_grid_size = 0.04
        num_rows = max(1, int(np.ceil(lat_span / optimal_grid_size)))
    
    if num_cols is None:
        # Aim for grids of ~0.04 degrees longitude (~3-4 km depending on latitude)
        optimal_grid_size = 0.04
        num_cols = max(1, int(np.ceil(lon_span / optimal_grid_size)))
    
    # Calculate actual grid step sizes
    lat_step = lat_span / num_rows
    lon_step = lon_span / num_cols
    
    # Generate grids
    grids = []
    grid_id = 1
    
    for row in range(num_rows):
        for col in range(num_cols):
            grid_ne_lat = ne_lat - (row * lat_step)
            grid_sw_lat = grid_ne_lat - lat_step
            grid_sw_long = sw_long + (col * lon_step)
            grid_ne_long = grid_sw_long + lon_step
            
            grids.append({
                'grid_id': grid_id,
                'ne_lat': round(grid_ne_lat, 5),
                'ne_long': round(grid_ne_long, 5),
                'sw_lat': round(grid_sw_lat, 5),
                'sw_long': round(grid_sw_long, 5)
            })
            grid_id += 1
    
    # Auto-generate output filename if not provided
    if output_file is None:
        output_file = f"../Grid-Coords-Files/grid_{region_name.lower().replace(' ', '_')}.xlsx"
    
    # Create output directory if needed
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    
    # Save to Excel
    df = pd.DataFrame(grids)

    # df.to_excel(output_file, index=False)
    
    # Print summary
    print("=" * 70)
    print(f"📍 GRID GENERATOR - {region_name}")
    print("=" * 70)
    print(f"\n📐 Input Bounds:")
    print(f"  NE Corner: ({ne_lat:.5f}, {ne_long:.5f})")
    print(f"  SW Corner: ({sw_lat:.5f}, {sw_long:.5f})")
    print(f"  Area Size: {lat_span:.3f}° lat × {lon_span:.3f}° lon")
    print(f"  Approx:    {lat_span * 111:.1f} km × {abs(lon_span * 85):.1f} km")
    
    print(f"\n✅ Generated Grid:")
    print(f"  Total Grids: {len(grids)}")
    print(f"  Layout: {num_rows} rows × {num_cols} columns")
    print(f"  Each Grid: {lat_step:.4f}° lat × {lon_step:.4f}° lon")
    print(f"  Approx:    {lat_step * 111:.2f} km × {abs(lon_step * 85):.2f} km")
    
    # print(f"\n📁 Saved to: {output_file}")
    # print(f"💡 Use in notebook: grid_filename = '{Path(output_file).name}'")
    
    # print(f"\n📊 Preview (first 5 grids):")
    # print(df.head(5).to_string(index=False))
    
    print("\n" + "=" * 70)
    
    return df




def segment_grid_file(grid_coords_df, region_name, rows_per_file=30):
    """
    Segment a grid file into multiple smaller files based on the number of rows.
    """
    num_rows = len(grid_coords_df)
    num_chunks = (num_rows + rows_per_file - 1) // rows_per_file

    for i in range(num_chunks):
        start_idx = i * rows_per_file
        end_idx = min((i + 1) * rows_per_file, num_rows)
        chunk = grid_coords_df.iloc[start_idx:end_idx, :]

        # Grid indexes are 1-based in the grid_id column
        first_grid_id = int(chunk.iloc[0]["grid_id"])
        last_grid_id = int(chunk.iloc[-1]["grid_id"])
        
        output_dir = f"../Grid-Coords-Files/{region_name}"
        os.makedirs(output_dir, exist_ok=True)
        filename = f"{output_dir}/{region_name}_full_safe_grid{first_grid_id}_to_grid{last_grid_id}_rows.xlsx"

        print(f"Saving rows {start_idx} to {end_idx - 1} as grids {first_grid_id}-{last_grid_id} to: {filename}")
        chunk.to_excel(filename, index=False)

In [2]:
def plot_grid_on_map(grid_coords_df, region_name="Region", show_labels=True, output_file=None):
    """
    Plot grid cells on an interactive map.
    
    Args:
        grid_coords_df: DataFrame with columns ['grid_id', 'ne_lat', 'ne_long', 'sw_lat', 'sw_long']
        region_name: Name of the region
        show_labels: Whether to show grid ID labels (default True)
        output_file: Where to save HTML (auto-generated if None)
    
    Returns:
        folium.Map object
    """
    import folium
    from folium import plugins
    
    # Calculate center of the region
    center_lat = (grid_coords_df['ne_lat'].max() + grid_coords_df['sw_lat'].min()) / 2
    center_long = (grid_coords_df['ne_long'].max() + grid_coords_df['sw_long'].min()) / 2
    
    # Create map
    m = folium.Map(
        location=[center_lat, center_long],
        zoom_start=12,
        tiles='OpenStreetMap'
    )
    
    print(f"🗺️  Creating map for {region_name}...")
    print(f"📍 Map center: ({center_lat:.5f}, {center_long:.5f})")
    print(f"📊 Total grids to plot: {len(grid_coords_df)}")
    print("=" * 70)
    
    # Add each grid cell as a rectangle
    for idx, row in grid_coords_df.iterrows():
        grid_id = row['grid_id']
        ne_lat = row['ne_lat']
        ne_long = row['ne_long']
        sw_lat = row['sw_lat']
        sw_long = row['sw_long']
        
        # Calculate grid dimensions
        lat_span = abs(ne_lat - sw_lat)
        lon_span = abs(ne_long - sw_long)
        
        # Define rectangle bounds
        bounds = [
            [sw_lat, sw_long],  # Southwest corner
            [ne_lat, ne_long]   # Northeast corner
        ]
        
        # Create popup with grid info
        popup_html = f"""
        <div style="font-family: Arial; font-size: 12px; min-width: 200px;">
            <b style="color: #2c3e50; font-size: 14px;">Grid {grid_id}</b>
            <hr style="margin: 5px 0;">
            <table style="width: 100%; border-collapse: collapse;">
                <tr>
                    <td style="padding: 3px;"><b>NE Corner:</b></td>
                    <td style="padding: 3px;">{ne_lat:.5f}, {ne_long:.5f}</td>
                </tr>
                <tr>
                    <td style="padding: 3px;"><b>SW Corner:</b></td>
                    <td style="padding: 3px;">{sw_lat:.5f}, {sw_long:.5f}</td>
                </tr>
                <tr>
                    <td style="padding: 3px;"><b>Size:</b></td>
                    <td style="padding: 3px;">{lat_span:.4f}° × {lon_span:.4f}°</td>
                </tr>
                <tr>
                    <td style="padding: 3px;"><b>Approx:</b></td>
                    <td style="padding: 3px;">{lat_span*111:.2f} km × {abs(lon_span*85):.2f} km</td>
                </tr>
            </table>
        </div>
        """
        
        # Add rectangle to map
        folium.Rectangle(
            bounds=bounds,
            color='blue',           # Border color
            fill=True,
            fillColor='blue',       # Fill color
            fillOpacity=0.15,       # Transparency (0.15 = 15% opaque)
            weight=2,               # Border thickness
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"Grid {grid_id}"
        ).add_to(m)
        
        # Add grid ID label at center (if enabled)
        if show_labels:
            center_lat_grid = (ne_lat + sw_lat) / 2
            center_long_grid = (ne_long + sw_long) / 2
            
            folium.Marker(
                location=[center_lat_grid, center_long_grid],
                icon=folium.DivIcon(html=f"""
                    <div style="
                        font-size: 10px;
                        font-weight: bold;
                        color: #1e3a8a;
                        text-align: center;
                        text-shadow: 1px 1px 2px white, -1px -1px 2px white, 1px -1px 2px white, -1px 1px 2px white;
                    ">
                        {grid_id}
                    </div>
                """)
            ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Add fullscreen button
    plugins.Fullscreen().add_to(m)
    
    # Auto-generate output filename if not provided
    if output_file is None:
        output_dir = f"Explore/HTML/{region_name}"
        os.makedirs(output_dir, exist_ok=True)
        output_file = f"{output_dir}/grid_visualization_before_disocvery_search.html"
    
    # Save map
    m.save(output_file)
    
    print(f"✅ Map created successfully!")
    print(f"📁 Saved to: {output_file}")
    print(f"💡 Open the HTML file in your browser to view the interactive map")
    print("=" * 70)
    
    return m


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

# # Example 1: Create grid and plot it
# if __name__ == "__main__":
#     # Create a sample grid for Ann Arbor
#     ne_lat, ne_long = 42.3314, -83.7430
#     sw_lat, sw_long = 42.2288, -83.7990
    
#     # Generate grid
#     grid_df = create_grid_from_bounds(
#         ne_lat=ne_lat,
#         ne_long=ne_long,
#         sw_lat=sw_lat,
#         sw_long=sw_long,
#         region_name="Ann-Arbor"
#     )
    
#     # Plot on map
#     map_obj = plot_grid_on_map(
#         grid_coords_df=grid_df,
#         region_name="Ann-Arbor",
#         show_labels=True  # Set to False to hide grid ID labels
#     )
    
#     print("\n🎉 Done! Check the HTML file to see your grid on the map.")


# Example 2: Plot an existing grid file
def plot_existing_grid(grid_file_path, region_name):
    """Load an existing grid file and plot it"""
    import pandas as pd
    
    grid_df = pd.read_excel(grid_file_path)
    map_obj = plot_grid_on_map(grid_df, region_name)
    return map_obj


# Example 3: Plot with custom output location
def plot_grid_custom(grid_df, region_name, output_path):
    """Plot grid with custom output path"""
    map_obj = plot_grid_on_map(
        grid_coords_df=grid_df,
        region_name=region_name,
        show_labels=True,
        output_file=output_path
    )
    return map_obj

# Chicago

In [155]:
chicago_full_safe = create_grid_from_bounds(
    ne_lat=42.02,
    ne_long=-87.52,
    sw_lat=41.64,
    sw_long=-87.94,
    region_name="Chicago_Full_Safe",
    # Force smaller grids:
    num_rows=15,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=17,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

chicago_full_safe.shape

📍 GRID GENERATOR - Chicago_Full_Safe

📐 Input Bounds:
  NE Corner: (42.02000, -87.52000)
  SW Corner: (41.64000, -87.94000)
  Area Size: 0.380° lat × 0.420° lon
  Approx:    42.2 km × 35.7 km

✅ Generated Grid:
  Total Grids: 255
  Layout: 15 rows × 17 columns
  Each Grid: 0.0253° lat × 0.0247° lon
  Approx:    2.81 km × 2.10 km



(255, 5)

In [5]:
rows_per_file = 30
num_rows = len(chicago_full_safe)
num_chunks = (num_rows + rows_per_file - 1) // rows_per_file

for i in range(num_chunks):
    start_idx = i * rows_per_file
    end_idx = min((i + 1) * rows_per_file, num_rows)
    chunk = chicago_full_safe.iloc[start_idx:end_idx, :]

    # Grid indexes are 1-based in the grid_id column
    first_grid_id = int(chunk.iloc[0]["grid_id"])
    last_grid_id = int(chunk.iloc[-1]["grid_id"])
    filename = f"../Grid-Coords-Files/Chicago/chicago_full_safe_grid{first_grid_id}_to_grid{last_grid_id}_rows.xlsx"

    print(f"Saving rows {start_idx} to {end_idx - 1} as grids {first_grid_id}-{last_grid_id} to: {filename}")
    chunk.to_excel(filename, index=False)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 119 as grids 91-120 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid91_to_grid120_rows.xlsx
Saving rows 120 to 149 as grids 121-150 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid121_to_grid150_rows.xlsx
Saving rows 150 to 179 as grids 151-180 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid151_to_grid180_rows.xlsx
Saving rows 180 to 209 as grids 181-210 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid181_to_grid210_rows.xlsx
Saving rows 210 to 239 as grids 211-240 to: ../Grid-Coords-Files/Chicago/chicago_full_safe_grid211_to_grid240_rows.xlsx
Saving rows 240 to 254 as grids 241-255 to: ../Grid-Coords-Files

# Sedona

In [156]:
sedona_full_safe = create_grid_from_bounds(
    ne_lat=34.88675,
    ne_long= -111.74344,
    sw_lat=34.77124,
    sw_long=-111.93187,
    region_name="Sedona_Full_Safe",
    # Force smaller grids:
    num_rows=5,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=6,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

sedona_full_safe.shape

📍 GRID GENERATOR - Sedona_Full_Safe

📐 Input Bounds:
  NE Corner: (34.88675, -111.74344)
  SW Corner: (34.77124, -111.93187)
  Area Size: 0.116° lat × 0.188° lon
  Approx:    12.8 km × 16.0 km

✅ Generated Grid:
  Total Grids: 30
  Layout: 5 rows × 6 columns
  Each Grid: 0.0231° lat × 0.0314° lon
  Approx:    2.56 km × 2.67 km



(30, 5)

In [25]:
segment_grid_file(
    grid_coords_df = sedona_full_safe,
    region_name = "Sedona", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Sedona/Sedona_full_safe_grid1_to_grid30_rows.xlsx


# New Jersey

In [157]:
new_jersey_full_safe = create_grid_from_bounds(
    ne_lat=40.86607,
    ne_long= -73.95019,
    sw_lat=40.64174,
    sw_long= -74.15993,
    region_name="New_Jersey_Full_Safe",
    # Force smaller grids:
    num_rows=10,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=10,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

new_jersey_full_safe.shape

📍 GRID GENERATOR - New_Jersey_Full_Safe

📐 Input Bounds:
  NE Corner: (40.86607, -73.95019)
  SW Corner: (40.64174, -74.15993)
  Area Size: 0.224° lat × 0.210° lon
  Approx:    24.9 km × 17.8 km

✅ Generated Grid:
  Total Grids: 100
  Layout: 10 rows × 10 columns
  Each Grid: 0.0224° lat × 0.0210° lon
  Approx:    2.49 km × 1.78 km



(100, 5)

In [28]:
segment_grid_file(
    grid_coords_df = new_jersey_full_safe,
    region_name = "New-Jersey", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/New-Jersey/New-Jersey_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/New-Jersey/New-Jersey_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/New-Jersey/New-Jersey_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 99 as grids 91-100 to: ../Grid-Coords-Files/New-Jersey/New-Jersey_full_safe_grid91_to_grid100_rows.xlsx


# Ann Arbor

In [158]:
ann_arbor_full_safe = create_grid_from_bounds(
    ne_lat = 42.44382,
    ne_long= -83.27921,
    sw_lat= 42.13855,
    sw_long= -84.0613,
    region_name="Ann_Arbor_Full_Safe",
    # Force smaller grids:
    num_rows=12,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=22,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

ann_arbor_full_safe.shape

📍 GRID GENERATOR - Ann_Arbor_Full_Safe

📐 Input Bounds:
  NE Corner: (42.44382, -83.27921)
  SW Corner: (42.13855, -84.06130)
  Area Size: 0.305° lat × 0.782° lon
  Approx:    33.9 km × 66.5 km

✅ Generated Grid:
  Total Grids: 264
  Layout: 12 rows × 22 columns
  Each Grid: 0.0254° lat × 0.0355° lon
  Approx:    2.82 km × 3.02 km



(264, 5)

In [49]:
segment_grid_file(
    grid_coords_df = ann_arbor_full_safe,
    region_name = "Ann-Arbor", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 119 as grids 91-120 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid91_to_grid120_rows.xlsx
Saving rows 120 to 149 as grids 121-150 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid121_to_grid150_rows.xlsx
Saving rows 150 to 179 as grids 151-180 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid151_to_grid180_rows.xlsx
Saving rows 180 to 209 as grids 181-210 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid181_to_grid210_rows.xlsx
Saving rows 210 to 239 as grids 211-240 to: ../Grid-Coords-Files/Ann-Arbor/Ann-Arbor_full_safe_grid211_to_grid240_rows.xlsx
Saving rows 240 to 263 as grids 

# Detroit

In [159]:
detroit_full_safe = create_grid_from_bounds(
    ne_lat = 42.45158,
    ne_long = -82.86293,
    sw_lat = 42.15246,
    sw_long = -83.28247,
    region_name="Detroit_Full_Safe",
    # Force smaller grids:
    num_rows=12,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=12,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

detroit_full_safe.shape

📍 GRID GENERATOR - Detroit_Full_Safe

📐 Input Bounds:
  NE Corner: (42.45158, -82.86293)
  SW Corner: (42.15246, -83.28247)
  Area Size: 0.299° lat × 0.420° lon
  Approx:    33.2 km × 35.7 km

✅ Generated Grid:
  Total Grids: 144
  Layout: 12 rows × 12 columns
  Each Grid: 0.0249° lat × 0.0350° lon
  Approx:    2.77 km × 2.97 km



(144, 5)

In [71]:
segment_grid_file(
    grid_coords_df = detroit_full_safe,
    region_name = "Detroit", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Detroit/Detroit_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Detroit/Detroit_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/Detroit/Detroit_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 119 as grids 91-120 to: ../Grid-Coords-Files/Detroit/Detroit_full_safe_grid91_to_grid120_rows.xlsx
Saving rows 120 to 143 as grids 121-144 to: ../Grid-Coords-Files/Detroit/Detroit_full_safe_grid121_to_grid144_rows.xlsx


# Napa

In [160]:
napa_full_safe = create_grid_from_bounds(
    ne_lat = 38.36081, 
    ne_long = -122.23466,
    sw_lat = 38.26787, 
    sw_long = -122.35024,
    region_name="Napa_Full_Safe",
    # Force smaller grids:
    num_rows=12,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=11,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

napa_full_safe.shape

📍 GRID GENERATOR - Napa_Full_Safe

📐 Input Bounds:
  NE Corner: (38.36081, -122.23466)
  SW Corner: (38.26787, -122.35024)
  Area Size: 0.093° lat × 0.116° lon
  Approx:    10.3 km × 9.8 km

✅ Generated Grid:
  Total Grids: 132
  Layout: 12 rows × 11 columns
  Each Grid: 0.0077° lat × 0.0105° lon
  Approx:    0.86 km × 0.89 km



(132, 5)

In [81]:
segment_grid_file(
    grid_coords_df = napa_full_safe,
    region_name = "Napa", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Napa/Napa_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Napa/Napa_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/Napa/Napa_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 119 as grids 91-120 to: ../Grid-Coords-Files/Napa/Napa_full_safe_grid91_to_grid120_rows.xlsx
Saving rows 120 to 131 as grids 121-132 to: ../Grid-Coords-Files/Napa/Napa_full_safe_grid121_to_grid132_rows.xlsx


# Sacramento

In [161]:
sacramento_full_safe = create_grid_from_bounds(
    ne_lat = 38.88221, 
    ne_long = -121.13507,
    sw_lat = 38.34569, 
    sw_long = -121.58825,
    region_name="Sacramento_Full_Safe",
    # Force smaller grids:
    num_rows=12,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=10,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

sacramento_full_safe.shape

📍 GRID GENERATOR - Sacramento_Full_Safe

📐 Input Bounds:
  NE Corner: (38.88221, -121.13507)
  SW Corner: (38.34569, -121.58825)
  Area Size: 0.537° lat × 0.453° lon
  Approx:    59.6 km × 38.5 km

✅ Generated Grid:
  Total Grids: 120
  Layout: 12 rows × 10 columns
  Each Grid: 0.0447° lat × 0.0453° lon
  Approx:    4.96 km × 3.85 km



(120, 5)

In [112]:
segment_grid_file(
    grid_coords_df = sacramento_full_safe,
    region_name = "Sacramento", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Sacramento/Sacramento_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Sacramento/Sacramento_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/Sacramento/Sacramento_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 119 as grids 91-120 to: ../Grid-Coords-Files/Sacramento/Sacramento_full_safe_grid91_to_grid120_rows.xlsx


# Stockton

In [162]:
stockton_full_safe = create_grid_from_bounds(
    ne_lat = 38.06015, 
    ne_long = -121.15206,
    sw_lat = 37.85221, 
    sw_long = -121.4088,
    region_name="Stockton_Full_Safe",
    # Force smaller grids:
    num_rows=8,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=8,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

stockton_full_safe.shape

📍 GRID GENERATOR - Stockton_Full_Safe

📐 Input Bounds:
  NE Corner: (38.06015, -121.15206)
  SW Corner: (37.85221, -121.40880)
  Area Size: 0.208° lat × 0.257° lon
  Approx:    23.1 km × 21.8 km

✅ Generated Grid:
  Total Grids: 64
  Layout: 8 rows × 8 columns
  Each Grid: 0.0260° lat × 0.0321° lon
  Approx:    2.89 km × 2.73 km



(64, 5)

In [118]:
segment_grid_file(
    grid_coords_df = stockton_full_safe,
    region_name = "Stockton", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Stockton/Stockton_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Stockton/Stockton_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 63 as grids 61-64 to: ../Grid-Coords-Files/Stockton/Stockton_full_safe_grid61_to_grid64_rows.xlsx


# San Fransico

In [163]:
san_fransico_full_safe = create_grid_from_bounds(
    ne_lat = 37.82754, 
    ne_long = -122.34648,
    sw_lat = 37.37219, 
    sw_long = -122.51539,
    region_name="San-Fransico_Full_Safe",
    # Force smaller grids:
    num_rows=17,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=5,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

san_fransico_full_safe.shape

📍 GRID GENERATOR - San-Fransico_Full_Safe

📐 Input Bounds:
  NE Corner: (37.82754, -122.34648)
  SW Corner: (37.37219, -122.51539)
  Area Size: 0.455° lat × 0.169° lon
  Approx:    50.5 km × 14.4 km

✅ Generated Grid:
  Total Grids: 85
  Layout: 17 rows × 5 columns
  Each Grid: 0.0268° lat × 0.0338° lon
  Approx:    2.97 km × 2.87 km



(85, 5)

In [127]:
segment_grid_file(
    grid_coords_df = san_fransico_full_safe,
    region_name = "San-Fransico", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/San-Fransico/San-Fransico_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/San-Fransico/San-Fransico_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 84 as grids 61-85 to: ../Grid-Coords-Files/San-Fransico/San-Fransico_full_safe_grid61_to_grid85_rows.xlsx


In [137]:
san_fransico_2_full_safe = create_grid_from_bounds(
    ne_lat = 37.58739, 
    ne_long = -122.12681,
    sw_lat = 37.36561, 
    sw_long = -122.33761,
    region_name="San-Fransico-2_Full_Safe",
    # Force smaller grids:
    num_rows=8,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=6,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

san_fransico_2_full_safe.shape

📍 GRID GENERATOR - San-Fransico-2_Full_Safe

📐 Input Bounds:
  NE Corner: (37.58739, -122.12681)
  SW Corner: (37.36561, -122.33761)
  Area Size: 0.222° lat × 0.211° lon
  Approx:    24.6 km × 17.9 km

✅ Generated Grid:
  Total Grids: 48
  Layout: 8 rows × 6 columns
  Each Grid: 0.0277° lat × 0.0351° lon
  Approx:    3.08 km × 2.99 km



(48, 5)

In [136]:
segment_grid_file(
    grid_coords_df = san_fransico_2_full_safe,
    region_name = "San-Fransico-2", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/San-Fransico-2/San-Fransico-2_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 47 as grids 31-48 to: ../Grid-Coords-Files/San-Fransico-2/San-Fransico-2_full_safe_grid31_to_grid48_rows.xlsx


In [143]:
san_fransico_3_full_safe = create_grid_from_bounds(
    ne_lat = 37.47837,
    ne_long = -121.97612,
    sw_lat = 37.21403, 
    sw_long = -122.15448,
    region_name="San-Fransico-3_Full_Safe",
    # Force smaller grids:
    num_rows=10,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=6,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

san_fransico_3_full_safe.shape

📍 GRID GENERATOR - San-Fransico-3_Full_Safe

📐 Input Bounds:
  NE Corner: (37.47837, -121.97612)
  SW Corner: (37.21403, -122.15448)
  Area Size: 0.264° lat × 0.178° lon
  Approx:    29.3 km × 15.2 km

✅ Generated Grid:
  Total Grids: 60
  Layout: 10 rows × 6 columns
  Each Grid: 0.0264° lat × 0.0297° lon
  Approx:    2.93 km × 2.53 km



(60, 5)

In [144]:
segment_grid_file(
    grid_coords_df = san_fransico_3_full_safe,
    region_name = "San-Fransico-3", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/San-Fransico-3/San-Fransico-3_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/San-Fransico-3/San-Fransico-3_full_safe_grid31_to_grid60_rows.xlsx


# San Jose

In [153]:
san_jose_full_safe = create_grid_from_bounds(
    ne_lat = 37.49716,
    ne_long =  -121.72838,
    sw_lat = 37.18503, 
    sw_long = -121.99381,
    region_name="San-Jose_Full_Safe",
    # Force smaller grids:
    num_rows=12,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=8,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

san_jose_full_safe.shape

📍 GRID GENERATOR - San-Jose_Full_Safe

📐 Input Bounds:
  NE Corner: (37.49716, -121.72838)
  SW Corner: (37.18503, -121.99381)
  Area Size: 0.312° lat × 0.265° lon
  Approx:    34.6 km × 22.6 km

✅ Generated Grid:
  Total Grids: 96
  Layout: 12 rows × 8 columns
  Each Grid: 0.0260° lat × 0.0332° lon
  Approx:    2.89 km × 2.82 km



(96, 5)

In [154]:
segment_grid_file(
    grid_coords_df = san_jose_full_safe,
    region_name = "San-Jose", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/San-Jose/San-Jose_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/San-Jose/San-Jose_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/San-Jose/San-Jose_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 95 as grids 91-96 to: ../Grid-Coords-Files/San-Jose/San-Jose_full_safe_grid91_to_grid96_rows.xlsx


# Fremont, Union City, CA

In [172]:
fremont_full_safe = create_grid_from_bounds(
    ne_lat = 37.62825,
    ne_long = -121.86942,
    sw_lat = 37.47665,
    sw_long = -122.16022,
    region_name="Fremont_Full_Safe",
    # Force smaller grids:
    num_rows=6,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=9,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

fremont_full_safe.shape

📍 GRID GENERATOR - Fremont_Full_Safe

📐 Input Bounds:
  NE Corner: (37.62825, -121.86942)
  SW Corner: (37.47665, -122.16022)
  Area Size: 0.152° lat × 0.291° lon
  Approx:    16.8 km × 24.7 km

✅ Generated Grid:
  Total Grids: 54
  Layout: 6 rows × 9 columns
  Each Grid: 0.0253° lat × 0.0323° lon
  Approx:    2.80 km × 2.75 km



(54, 5)

In [173]:
segment_grid_file(
    grid_coords_df = fremont_full_safe,
    region_name = "Fremont", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Fremont/Fremont_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 53 as grids 31-54 to: ../Grid-Coords-Files/Fremont/Fremont_full_safe_grid31_to_grid54_rows.xlsx


# Oakland, CA

In [180]:
oakland_full_safe = create_grid_from_bounds(
    ne_lat = 37.87796, 
    ne_long = -121.99855,
    sw_lat = 37.58688,
    sw_long = -122.33633,
    region_name="Oakland_Full_Safe",
    # Force smaller grids:
    num_rows=11,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=10,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

oakland_full_safe.shape

📍 GRID GENERATOR - Oakland_Full_Safe

📐 Input Bounds:
  NE Corner: (37.87796, -121.99855)
  SW Corner: (37.58688, -122.33633)
  Area Size: 0.291° lat × 0.338° lon
  Approx:    32.3 km × 28.7 km

✅ Generated Grid:
  Total Grids: 110
  Layout: 11 rows × 10 columns
  Each Grid: 0.0265° lat × 0.0338° lon
  Approx:    2.94 km × 2.87 km



(110, 5)

In [181]:
segment_grid_file(
    grid_coords_df = oakland_full_safe,
    region_name = "Oakland", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Oakland/Oakland_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Oakland/Oakland_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/Oakland/Oakland_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 109 as grids 91-110 to: ../Grid-Coords-Files/Oakland/Oakland_full_safe_grid91_to_grid110_rows.xlsx


# Above San Franciso


In [13]:
above_san_franciso_full_safe = create_grid_from_bounds(
    ne_lat = 38.1447, 
    ne_long = -121.89342,
    sw_lat = 37.81021,
    sw_long = -122.63087,
    region_name="Above-San-Franciso_Full_Safe",
    num_rows=13,    
    num_cols=21,    
)

above_san_franciso_full_safe.shape

📍 GRID GENERATOR - Above-San-Franciso_Full_Safe

📐 Input Bounds:
  NE Corner: (38.14470, -121.89342)
  SW Corner: (37.81021, -122.63087)
  Area Size: 0.334° lat × 0.737° lon
  Approx:    37.1 km × 62.7 km

✅ Generated Grid:
  Total Grids: 273
  Layout: 13 rows × 21 columns
  Each Grid: 0.0257° lat × 0.0351° lon
  Approx:    2.86 km × 2.98 km



(273, 5)

In [15]:
segment_grid_file(
    grid_coords_df = above_san_franciso_full_safe,
    region_name = "Above-San-Franciso", 
    rows_per_file = 30
)

Saving rows 0 to 29 as grids 1-30 to: ../Grid-Coords-Files/Above-San-Franciso/Above-San-Franciso_full_safe_grid1_to_grid30_rows.xlsx
Saving rows 30 to 59 as grids 31-60 to: ../Grid-Coords-Files/Above-San-Franciso/Above-San-Franciso_full_safe_grid31_to_grid60_rows.xlsx
Saving rows 60 to 89 as grids 61-90 to: ../Grid-Coords-Files/Above-San-Franciso/Above-San-Franciso_full_safe_grid61_to_grid90_rows.xlsx
Saving rows 90 to 119 as grids 91-120 to: ../Grid-Coords-Files/Above-San-Franciso/Above-San-Franciso_full_safe_grid91_to_grid120_rows.xlsx
Saving rows 120 to 149 as grids 121-150 to: ../Grid-Coords-Files/Above-San-Franciso/Above-San-Franciso_full_safe_grid121_to_grid150_rows.xlsx
Saving rows 150 to 179 as grids 151-180 to: ../Grid-Coords-Files/Above-San-Franciso/Above-San-Franciso_full_safe_grid151_to_grid180_rows.xlsx
Saving rows 180 to 209 as grids 181-210 to: ../Grid-Coords-Files/Above-San-Franciso/Above-San-Franciso_full_safe_grid181_to_grid210_rows.xlsx
Saving rows 210 to 239 as grid

# Save Map

In [16]:
map_obj = plot_grid_on_map(
        grid_coords_df=above_san_franciso_full_safe,
        region_name="Above-San-Franciso",
        show_labels=True  # Set to False to hide grid ID labels
    )


🗺️  Creating map for Above-San-Franciso...
📍 Map center: (37.97745, -122.26215)
📊 Total grids to plot: 273
✅ Map created successfully!
📁 Saved to: Explore/HTML/Above-San-Franciso/grid_visualization_before_disocvery_search.html
💡 Open the HTML file in your browser to view the interactive map


# For Testing

In [98]:
test_full_safe = create_grid_from_bounds(
    ne_lat = 38.29885, 
    ne_long = -122.28720,
    sw_lat = 38.29111, 
    sw_long = -122.29770,
    region_name="Test_Full_Safe",
    # Force smaller grids:
    num_rows=2,     # Was 10, now 15 (smaller lat span per grid)
    num_cols=1,     # Was 11, now 17 (smaller lon span per grid)
    # Total: 15 × 17 = 255 grids (instead of 110)
)

test_full_safe

📍 GRID GENERATOR - Test_Full_Safe

📐 Input Bounds:
  NE Corner: (38.29885, -122.28720)
  SW Corner: (38.29111, -122.29770)
  Area Size: 0.008° lat × 0.011° lon
  Approx:    0.9 km × 0.9 km

✅ Generated Grid:
  Total Grids: 2
  Layout: 2 rows × 1 columns
  Each Grid: 0.0039° lat × 0.0105° lon
  Approx:    0.43 km × 0.89 km



,grid_id,ne_lat,ne_long,sw_lat,sw_long
0,1,38.29885,-122.2872,38.29498,-122.2977
1,2,38.29498,-122.2872,38.29111,-122.2977


In [99]:
segment_grid_file(
    grid_coords_df = test_full_safe,
    region_name = "Test", 
    rows_per_file = 30
)

Saving rows 0 to 1 as grids 1-2 to: ../Grid-Coords-Files/Test/Test_full_safe_grid1_to_grid2_rows.xlsx
